In [1]:
import sys
import csv
import os
import pickle
import re
from datetime import date, datetime, timedelta

from pymongo import MongoClient
import psycopg2 as pg
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas.io.sql as pd_sql

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction import text
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler, MinMaxScaler, minmax_scale, RobustScaler

# from nltk.stem.porter import PorterStemmer
from textblob import TextBlob
from textblob.taggers import NLTKTagger
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# nltk.download('vader_lexicon')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')

In [2]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [3]:
nltk_tagger = NLTKTagger()
pd.set_option('display.max_colwidth', -1)

In [4]:
# NMF topics
num_topics = 10
num_keywords = 8

In [5]:
with open('../data/df_cities.pkl', "rb") as fp:
    df_cities = pickle.load(fp)
with open('../data/df_states.pkl', "rb") as fp:
    df_states = pickle.load(fp)

In [6]:
client = MongoClient()
db = client.r

db.posts.estimated_document_count()

140893

In [7]:
# TESTING
today = datetime.today()
lastyear = today - timedelta(days = 60)
# Extract posts from MongoDB to list, DataFrame:
posts = list(db.posts.find({"post_date": {"$gt": lastyear}}))
df_posts = pd.DataFrame(posts)

In [8]:
# Extract posts from MongoDB to list, DataFrame:
# posts = list(db.posts.find())
# df_posts = pd.DataFrame(posts)

In [9]:
# word count
df_posts['count'] = df_posts['selftext'].str.count(' ') + 1
df_posts['count'].quantile([0.05,0.5,0.95])

0.05    13.0 
0.50    48.0 
0.95    199.0
Name: count, dtype: float64

In [10]:
df_posts['post_id'].count()

63687

In [11]:
# remove duplicates ie spam, daily/weekly notices etc. 
df_posts = df_posts.drop_duplicates(subset='selftext')

min_word_cnt = 10
max_word_cnt = 1500

# remove extra-short and extra-long posts
df_posts = df_posts[(df_posts['count'] >= min_word_cnt) & (df_posts['count'] < max_word_cnt)]

# remove spam posts contains at least 5 consecutive words
df_posts = df_posts[~df_posts['selftext'].str.contains(r'\b(\w+)(\s+\1){4,}\b', r'\1', flags=re.IGNORECASE)]

/home/adam/anaconda3/envs/metis/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [12]:
df_posts['post_id'].count()

60468

In [14]:
custom_stop_word_list = ['area', 'thanks', 'place', 'state', 'people', 'time', 'year',
                        'day, city', 'town', 'week', 'question', 'county', 'said',
                        'thank', 'reddit', 'ave', 'really', 'hey', 'way', 'lot',
                        'thing', 'don', 'hour', 'idea', 'option', 'wa', 'does', 'ha',
                        'use', 'like', 'number', 'didn', 'doesn', 'car car', 'google',
                        'sub', 'blah', 'mod', 'lol', 'hello', 'month', 'issue',
                        'location', 'minute', 'today', 'example', 'sunday',
                        'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday',
                        'information', 'info', 'subreddit', 'wiki', 'january', 'february',
                        'march', 'april', 'may', 'june', 'july', 'august', 'september',
                        'october', 'november', 'december', 'great', 'new', 'bos', 'thx',
                        'shit', 'penis', 'couldn', 'fuck', 'just', 'today', 'tomorrow',
                        'sort', 'item', 'anybody', 'list', 'post', 'page', 'dont', 'img',
                        'wouldn', 'would', 'redditors']

custom_stop_words = ' '.join(custom_stop_word_list)
blob = TextBlob(custom_stop_words)
lemmatized_custom_stop_word_list = blob.words.lemmatize() # lemmatize all custom stop words
stop_words = ENGLISH_STOP_WORDS.union(lemmatized_custom_stop_word_list)

In [30]:
lemmatized_custom_stop_word_list

WordList(['area', 'thanks', 'place', 'state', 'people', 'time', 'year', 'day', 'city', 'town', 'week', 'question', 'county', 'said', 'thank', 'reddit', 'ave', 'really', 'hey', 'way', 'lot', 'thing', 'don', 'hour', 'idea', 'option', 'wa', 'doe', 'ha', 'use', 'like', 'number', 'didn', 'doesn', 'car', 'car', 'google', 'sub', 'blah', 'mod', 'lol', 'hello', 'month', 'issue', 'location', 'minute', 'today', 'example', 'sunday', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'information', 'info', 'subreddit', 'wiki', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'great', 'new', 'bos', 'thx', 'shit', 'penis', 'couldn', 'fuck', 'just', 'today', 'tomorrow', 'sort', 'item', 'anybody', 'list', 'post', 'page', 'dont', 'img', 'wouldn', 'would', 'redditors'])

In [15]:
def preprocess_text(post):
    """This is run before the sentiment analysis"""
    
    # remove | words |
    regex_pat = re.compile(r'\|.+?\|', flags=re.IGNORECASE)
    post = re.sub(regex_pat, '', post)

    # remove { words }
    regex_pat = re.compile(r'{.+?}', flags=re.IGNORECASE)
    post = re.sub(regex_pat, '', post)

    # remove ( words )
    regex_pat = re.compile(r'\(.+?\)', flags=re.IGNORECASE)
    post = re.sub(regex_pat, '', post)

    # remove [ words ]
    regex_pat = re.compile(r'\[.+?\]', flags=re.IGNORECASE)
    post = re.sub(regex_pat, '', post)

    # remove links
    regex_pat = re.compile(r'https?:\/\/.*[\r\n]*', flags=re.IGNORECASE)
    post = re.sub(regex_pat, '', post)
    
    # remove emails
    regex_pat = re.compile(r'\S*@\S*\s?', flags=re.IGNORECASE)
    post = re.sub(regex_pat, '', post)
        
    # remove any digits or words that start with digits ie 19th
    regex_pat = re.compile(r'\b\D?\d.*?\b', flags=re.IGNORECASE)
    post = re.sub(regex_pat, ' ', post)

    # remove words that start # ie #x200b
    regex_pat = re.compile(r'#.*\b', flags=re.IGNORECASE)
    post = re.sub(regex_pat, ' ', post)

    post = post.replace('&amp', 'and').replace('&nbsp', ' ')
    post = post.replace('’', "'").replace('‘', "'").replace('“', '"').replace('”', '”')
    
    # remove special stop words
    regex_pat = re.compile(r'update:|wibta|email|gmail|tldr| \
                             nbsp|\b.+_.+\b|\baaa.+\b|\bbbb.+\b', flags=re.IGNORECASE)
    post = re.sub(regex_pat, '', post)
    
    # collapse spaces
    regex_pat = re.compile(r'\s+', flags=re.IGNORECASE)
    post = re.sub(regex_pat, ' ', post)
    
    return post

In [16]:
def get_topics(nmf_model, num_keywords):
    num_topics = nmf_model.components_.shape[0]
    topics_dict = {}
    for ix, topic in enumerate(nmf_model.components_):
        topics_dict[ix] = ", ".join([count_vectorizer.get_feature_names()[i] 
                                     for i in topic.argsort()[:-num_keywords - 1:-1]])
    return topics_dict

In [17]:
# Preprocess the posts:
df_posts['selftext_preprocessed'] = df_posts['selftext'].apply(preprocess_text)

# Generate sentiment score (progress_apply uses tqdm)
sid = SentimentIntensityAnalyzer()
df_posts['sentiment_all'] = df_posts['selftext_preprocessed'].progress_apply(lambda x: sid.polarity_scores(x))

# extract compound score to new column
df_posts['sentiment_compound'] = df_posts['sentiment_all'].progress_apply(lambda x: x.get('compound'))

# pickle.dump(df_posts, open(f"data/df_posts_sent_pp.pkl", "wb"))

# comment out above 3 commands and use pickle file to save time:
# with open('data/df_posts_sent_pp.pkl', "rb") as fp:
#     df_posts = pickle.load(fp)

In [18]:
connection_args = {
    'host': os.getenv('POSTGRES_HOST'),
    'user': os.getenv('POSTGRES_USER'),
    'dbname': os.getenv('POSTGRES_RP_DB'),
    'port': os.getenv('POSTGRES_PORT')
}

connection = pg.connect(**connection_args)

In [19]:
# Extra step needed to calculate state sentiment since all cities within that state should be inclued as well
df_cities_tmp = df_cities[['city_sub', 'state_fip']]
df_cities_tmp = df_cities_tmp.rename(columns={'city_sub': 'subreddit'})
df_states_tmp = df_states[['state_sub', 'state_fip']]
df_states_tmp = df_states_tmp.rename(columns={'state_sub': 'subreddit'})
df_posts_tmp = df_posts[['subreddit', 'geo_type', 'sentiment_compound']]
df_posts_cities = pd.merge(df_posts_tmp, df_cities_tmp, on='subreddit')
df_posts_states = pd.merge(df_posts_tmp, df_states_tmp, on='subreddit')

frames = [df_posts_cities, df_posts_states]
df_posts_sent_tmp = pd.concat(frames)

In [20]:
# Create sentiment df for each subreddit
df_sent = df_posts.groupby('subreddit') \
                    .agg({'sentiment_compound':'mean', 'geo_type':'first'}) \
                    .reset_index()
sent_dict = df_sent.to_dict('records')

In [21]:
# Create sentiment df for each subreddit
df_sent_cities = df_posts_cities.groupby('subreddit') \
                    .agg({'sentiment_compound':'mean'}) \
                    .reset_index()
sent_dict_cities = df_sent_cities.to_dict('records')

In [22]:
# special case for states
df_sent_states = df_posts_sent_tmp.groupby('state_fip') \
                    .agg({'sentiment_compound':'mean'}) \
                    .reset_index()
sent_dict_states = df_sent_states.to_dict('records')

In [23]:
# df_sent_states = df_sent[df_sent['geo_type'] == 'state']
# df_sent_cities = df_sent[df_sent['geo_type'] == 'city']
quant_states_25 = round(df_sent_states.quantile(.25).values[1], 2)
quant_states_75 = round(df_sent_states.quantile(.75).values[1], 2)
quant_cities_25 = round(df_sent_cities.quantile(.25).values[0], 2)
quant_cities_75 = round(df_sent_cities.quantile(.75).values[0], 2)

In [23]:
# insert sentiments scores into Postgres
cursor = connection.cursor()

for row in sent_dict_states:
    state_fip = row['state_fip']
    sentiment_compound = round(row['sentiment_compound'], 2)

    if sentiment_compound > quant_states_75:
        sentiment_rating = 'pos'
    elif sentiment_compound < quant_states_25:
        sentiment_rating = 'neg'
    else:
        sentiment_rating = 'neu' # neutral
    
#     print(state_fip, sentiment_compound, sentiment_rating, quant_states_75, quant_states_25)

    query =  "UPDATE states SET sentiment_compound = %s, sentiment_rating = %s WHERE state_fip = %s"
    
    data = (sentiment_compound, sentiment_rating, state_fip)       
    cursor.execute(query, data)
    
    connection.commit()

cursor.close()

In [24]:
# insert sentiments scores into Postgres
cursor = connection.cursor()

for row in sent_dict_cities:
    subreddit = row['subreddit']
    sentiment_compound = round(row['sentiment_compound'], 2)

    if sentiment_compound > quant_cities_75:
        sentiment_rating = 'pos'
    elif sentiment_compound < quant_cities_25:
        sentiment_rating = 'neg'
    else:
        sentiment_rating = 'neu' # neutral

    query =  "UPDATE cities SET sentiment_compound = %s, sentiment_rating = %s WHERE city_sub = %s"
    
    data = (sentiment_compound, sentiment_rating, subreddit)       
    cursor.execute(query, data)
    
    connection.commit()

cursor.close()

In [25]:
def compute_ascore(geo):
    cursor = connection.cursor()

    scaler = MinMaxScaler()
    rscaler = RobustScaler()
    scorescaler = MinMaxScaler(feature_range=(50, 100))
    
    if geo == 'states':
        table = 'states'
        sub_field = 'state_sub'
    else:
        table = 'cities'
        sub_field = 'city_sub'
    
    query = f'SELECT {sub_field}, pop_2018, median_hh_income, sentiment_compound FROM {table}'
    df_metrics = pd_sql.read_sql(query, connection)
    
    df_metrics['pop_2018_scaled'] = scaler.fit_transform(df_metrics[['pop_2018']])
    df_metrics['median_hh_income_scaled'] = rscaler.fit_transform(df_metrics[['median_hh_income']])
    df_metrics['sentiment_compound_scaled'] = scaler.fit_transform(df_metrics[['sentiment_compound']])
    
    df_metrics['ascore_raw'] = (
                                   (df_metrics['pop_2018_scaled'] * 0.4)
                                   + 
                                   (df_metrics['median_hh_income_scaled'] * 0.5)
                                   + 
                                   (df_metrics['sentiment_compound'] * 0.1)
                                   * 100
                                   )
    
    df_metrics['ascore'] = scorescaler.fit_transform(df_metrics[['ascore_raw']])
    df_metrics['ascore'] = df_metrics['ascore'].round(1)

    for row in df_metrics.itertuples():
#         print(row[1], row.ascore)
        
        query =  f'UPDATE {table} SET ascore = %s WHERE {sub_field} = %s'
        data = (row.ascore, row[1])       
        cursor.execute(query, data)
        connection.commit()
    cursor.close()

compute_ascore('states')
compute_ascore('cities')

In [24]:
def clean_text(post):
    """Run after sentiment analysis"""
    blob = TextBlob(post)
    
    # delete 1 and 2 letter words
    regex_pat = re.compile(r'\b\w{1,2}\b', flags=re.IGNORECASE)
    post = re.sub(regex_pat, ' ', post)

    # remove quotes
    regex_pat = re.compile(r'[\'"]', flags=re.IGNORECASE)
    post = re.sub(regex_pat, '', post)
    
    # set textblob again with cleaned words:
    blob = TextBlob(post, pos_tagger=nltk_tagger)
    
    # lemmatize, lowercase, and only include nouns and proper nouns
    words = [token.lemmatize()
                  .lower() for token, (_, pos) in zip(blob.words, blob.tags)
                  if pos in ['NN', 'NNS', 'NNP', 'NNPS']]
    
    return ' '.join(words)

In [25]:
# df_states = df_states.drop(['sentiment_compound_x', 'ascore_x', 'sentiment_compound_y', 'ascore_y'], axis=1)

In [26]:
df_posts['selftext_final'] = df_posts['selftext_preprocessed'].progress_apply(clean_text)

# pickle.dump(df_posts, open(f"data/df_posts_final.pkl", "wb"))

# comment out above 3 commands and use pickle file to save time:
# with open('data/df_posts_final.pkl', "rb") as fp:
#     df_posts = pickle.load(fp)

In [29]:
df_posts['selftext_final'][df_posts['selftext_final'].str.contains('does')]

11       situation court week something before does difference offense degree                                                                                                                                           
16       does anyone place appliance fridge month something financing purchase thanks                                                                                                                                   
20       sorry doesn belong fun night interest friend serpentina selena birthday drag show cocktails akron show fun drink                                                                                               
25       found porch mom baby heard cardinal animal control something google nest tree doesn fly wonder feel prey mom baby google                                                                                       
29       therapist job service don insurance job does anyone area client insurance price                                            

In [29]:
cursor = connection.cursor()

query_topics = "TRUNCATE topics RESTART IDENTITY CASCADE" # CASCADE needed due to foreign key
query_kw = "TRUNCATE keywords RESTART IDENTITY CASCADE"
query_geo = "TRUNCATE topics_geo RESTART IDENTITY CASCADE"
query_tkw = "TRUNCATE topics_keywords RESTART IDENTITY CASCADE"
cursor.execute(query_topics)
cursor.execute(query_kw)
cursor.execute(query_geo)
cursor.execute(query_tkw)
connection.commit()
cursor.close()

In [30]:
cursor = connection.cursor()

query =  "INSERT INTO models (extract_date) VALUES (now()) RETURNING model_id"
cursor.execute(query)
model_id = cursor.fetchone()[0]
connection.commit()
cursor.close()

In [31]:
cursor = connection.cursor()

# query =  "INSERT INTO topics (topic) VALUES ('Topic') RETURNING topic_id"
# cursor.execute(query)
# topic_id = cursor.fetchone()[0]
# connection.commit()

for location in sent_dict:
    sub = location['subreddit']
    posts_final = df_posts['selftext_final'][df_posts['subreddit'] == sub]
    
    if len(df_states[df_states['state_sub'] == sub]):
        state_name = df_states.loc[df_states['state_sub'] == sub, 'state_name'].values[0].lower()
#         print(state_name)
        # it's necessary to lemmatize the word otherwise:
        # UserWarning: Your stop_words may be inconsistent with your preprocessing.
        blob = TextBlob(state_name)
        state_tokens = blob.words.lemmatize()
        
        # Tailor stop word for each location
        stop_words_tot = stop_words.union(state_tokens)
        geo_type = 'state'

        query =  "SELECT state_id FROM states WHERE state_sub = %s"
        data = (sub,) # tuple required for cursor.execute
        cursor.execute(query, data)
        state_id = cursor.fetchone()[0]
        geo_id = state_id
    else:
        city_short = df_cities.loc[df_cities['city_sub'] == sub, 'city_short'].values[0].lower()
#         print(city_short)
        # it's necessary to lemmatize the word otherwise:
        # UserWarning: Your stop_words may be inconsistent with your preprocessing.
        blob = TextBlob(city_short)
        city_tokens = blob.words.lemmatize()
        stop_words_tot = stop_words.union(city_tokens)
        geo_type = 'city'
        query =  "SELECT city_id FROM cities WHERE city_sub = %s"
        data = (sub,) # comma will turn it into a list which is required in cursor.execute
        cursor.execute(query, data)
        city_id = cursor.fetchone()[0]
        geo_id = city_id
    
    connection.commit()

    count_vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=stop_words_tot)    
    doc_word = count_vectorizer.fit_transform(posts_final)
    nmf_model = NMF(num_topics)
    nmf_model.fit_transform(doc_word)

    topics_dict = get_topics(nmf_model, num_keywords)
#     print(sub)
#     print(topics_dict)
    for k, v in topics_dict.items():
        keywords = v.split(', ') # convert keywords to list
        
        query =  "INSERT INTO topics (topic) VALUES ('Topic') RETURNING topic_id"
        cursor.execute(query)
        topic_id = cursor.fetchone()[0]
        connection.commit()
        
        for word in keywords:
            if len(word) <= 30: # do not include any word over 30 characters
                # check whether keyword exists and if so, get the id to use
                query = "SELECT keyword_id FROM keywords WHERE keyword = %s"
                data = (word,)
                cursor.execute(query, data)
                row_kw = cursor.fetchone()
                if row_kw is not None:
                    keyword_id = row_kw[0] # keyword exists
                else:
                    query_kw =  "INSERT INTO keywords (keyword) VALUES (%s) RETURNING keyword_id"
#                     print(word)
                    data_kw = (word,)
                    cursor.execute(query_kw, data_kw)   
                    keyword_id = cursor.fetchone()[0]                

                query_kw =  "INSERT INTO topics_keywords (topic_id, keyword_id) VALUES (%s, %s)"
                data_kw = (topic_id, keyword_id)
                cursor.execute(query_kw, data_kw)   

        # geo_type: city or state
        # geo_id: city_id or state_id
        query =  "INSERT INTO topics_geo (geo_type, geo_id, topic_id, model_id) VALUES (%s, %s, %s, %s)"
        data = (geo_type, geo_id, topic_id, model_id)
        cursor.execute(query, data)
            
        connection.commit()
cursor.close()

/home/adam/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/decomposition/nmf.py:113: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(res * 2)
/home/adam/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['alene'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/adam/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/decomposition/nmf.py:113: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(res * 2)


In [32]:
cursor = connection.cursor()
query =  """
         UPDATE keywords AS kw
         SET num_geo = sq.count FROM (
         SELECT count(tk.keyword_id) AS count, tk.keyword_id 
         FROM topics_keywords as tk 
         INNER JOIN keywords as k on k.keyword_id = tk.keyword_id
         GROUP BY tk.keyword_id
         ) AS sq
         WHERE kw.keyword_id = sq.keyword_id
         """
cursor.execute(query)
connection.commit()
cursor.close()

In [33]:
# EDA exploration used to create topic dictionary
# broad topics at top, narrow topics below which will override the broad ones
topics_dict = [
                   {'Work': ['work', 'job', 'management', 'employer', 'office']},
                   {'Entertainment': ['museum', 'play', 'music', 'fun', 'game', 'theater', 'theatre', 
                                      'comedy', 'movie']},
                   {'Housing': ['housing', 'apartment', 'lease', 'mortgage', 'rent', 'tenant', 
                                'rent', 'household', 'condo', 'hoa', 'lease', 'landlord']},
                   {'Vehicles': ['vehicle', 'suv', 'car', 'truck']},
                   {'Transportation': ['transportation', 'car', 'road', 'train', 'highway', 'parking', 
                                       'truck', 'plane', 'traffic', 'passenger', 'driver', 'driving']},
                   {'Religion': ['religion', 'church', 'christ', 'jesus', 'lord']},
                   {'Public Transit': ['public', 'train', 'route', 'bus', 'passenger', 'subway', 
                                         'line', 'transfer', 'transit', 'muni', 'bart']},
                   {'Travel': ['travel', 'adventure', 'pack', 'camera']},
                   {'Government': ['government', 'council', 'ordinance', 'complaint']},
                   {'School': ['school', 'campus', 'class', 'student', 'transfer', 'college']},
                   {'Food': ['food', 'grocery', 'store', 'sandwich']},
                   {'Restaurants': ['restaurant', 'seafood', 'pizza', 'eat', 'taco', 'service']},
                   {'College': ['college', 'university', 'engineering']},
                   {'Medicine': ['vaccine', 'doctor', 'operation', 'hospital']},
                   {'Crime': ['crime', 'property crime', 'theft', 'robbery', 'police', 'abuse', 'homicide',
                              'suspect', 'offender']},
                   {'Drugs': ['drug', 'drug problem', 'violation', 'order', 'substance']},
                   {'Homeless': ['homeless', 'problem', 'shelter', 'policy', 'depression']},
                   {'Jobs': ['career', 'job', 'resume', 'company', 'service', 'interview']},
                   {'Beaches': ['beach', 'sea', 'pier', 'island']},
                   {'Pizza': ['pizza', 'pepperoni', 'topping', 'pizza hut']},
                   {'Gaming': ['game', 'magic', 'video game', 'twitch']},
                   {'Pets': ['pet', 'cat', 'dog', 'fish', 'shelter', 'animal', 'breed', 'park', 'owner',
                             'rabbit']},
                   {'Service Animals': ['animal', 'service animal', 'dog', 'blind']},
                   {'Business': ['industry', 'business', 'meetup', 'job']},
                   {'Night Life': ['night', 'bar', 'drink', 'downtown']},
                   {'Ridesharing': ['taxi', 'uber', 'lyft', 'jump', 'ride']},
                   {'Co-Living': ['apartment', 'share', 'room', 'roommate']},
                   {'Elections': ['election', 'volunteer', 'campaign', 'gerrymandering', 'candidate']},
                   {'Politics': ['politics', 'republican', 'democrat', 'trump']},
                   {'Cycling': ['cycling', 'bike', 'lane', 'bicycle', 'ride', 'biker', 'bikers',
                                'bike lane', 'cyclist', 'rider']},
                   {'Sports': ['sport', 'basketball', 'baseball', 'football', 'soccer', 'ballpark', 
                               'tennis', 'season', 'team', 'game']},
                   {'Bars': ['alcohol', 'license', 'owner', 'pub', 'brew', 'beer', 'draft']},
                   {'Cafes': ['coffee', 'tea', 'bean']},
                   {'Health Care': ['health', 'health care', 'coverage']},
                   {'Salons': ['hair', 'haircut', 'color', 'nails', 'facial', 'barber']},
                   {'Reading': ['reading', 'book', 'library']},
                   {'Music': ['music', 'musician', 'venue']},
                   {'Desserts': ['dessert', 'ice cream', 'cake', 'pie', 'ice', 'cream']},
                   {'LGBTQ': ['LGBT', 'queer', 'gay', 'drag', 'pride', 'transgender']},
                   {'Air Transportation': ['air', 'jet', 'aircraft', 'plane']},
                   {'Parking': ['parking', 'garage']},
                   {'Tatoos': ['tatoos', 'artist']},
                   {'Internet Service': ['router', 'speed', 'xfinity', 'verizon', 'modem', 'router modem']}
              ]

In [34]:
threshold = .2
min_kw_topic = threshold * num_topics

def assign_topics():
    """Loop through topics keywords and assign topics from topics_dict if minimum keyword 
       match threshold is met"""
    
    cursor = connection.cursor()

    query = """SELECT tk.topic_id, string_agg(k.keyword, '-:-') AS keyword_list_db 
    FROM keywords AS k 
    INNER JOIN topics_keywords AS tk ON tk.keyword_id = k.keyword_id 
    INNER JOIN topics_geo AS tg ON tg.topic_id = tk.topic_id 
    LEFT JOIN cities AS c ON c.city_id = tg.geo_id AND tg.geo_type = 'city'
    LEFT JOIN states AS s ON s.state_id = tg.geo_id AND tg.geo_type = 'state'
    GROUP BY tg.geo_id, tk.topic_id"""
    
    cursor.execute(query)
    result = cursor.fetchall()
    connection.commit()
    
    for topic_id, keywords_db in result:
        keywords_list_db = keywords_db.split('-:-')

        # loop through defined topics
        for row in topics_dict:
            for topic, keywords_list in row.items():
                keyword_matches = len(set(keywords_list_db) & set(keywords_list))

                if keyword_matches >= min_kw_topic:
                    query =  "UPDATE topics SET topic = %s WHERE topic_id = %s"
                    data = (topic, topic_id)
                    cursor.execute(query, data)

                    connection.commit()
assign_topics()

In [35]:
cursor = connection.cursor()

query =  """INSERT INTO topics_archive (topic, geo_type, geo_id, topic_id, model_id)
            (SELECT t.topic, tg.geo_type, tg.geo_id, t.topic_id, tg.model_id 
             FROM topics as t
             INNER JOIN topics_geo AS tg ON tg.topic_id = t.topic_id
             WHERE t.topic != 'Topic'
            )"""
cursor.execute(query)
connection.commit()

query =  """INSERT INTO states_archive (sentiment_compound, sentiment_rating, ascore, state_id, model_id)
            (SELECT DISTINCT s.sentiment_compound, s.sentiment_rating, s.ascore, s.state_id, tg.model_id
             FROM states AS s
             INNER JOIN topics_geo AS tg ON tg.geo_id = s.state_id AND tg.geo_type = 'state'
            )"""
cursor.execute(query)
connection.commit()

query =  """INSERT INTO cities_archive (ascore, sentiment_compound, sentiment_rating, city_id, state_id, model_id)
            (SELECT DISTINCT c.ascore, c.sentiment_compound, c.sentiment_rating, c.city_id, c.state_id, tg.model_id
             FROM cities AS c
             INNER JOIN topics_geo AS tg ON tg.geo_id = c.state_id AND tg.geo_type = 'city'
            )"""
cursor.execute(query)
connection.commit()

cursor.close()